# Preparation

In [4]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 27

## Simulation setup

### Simulation params setup

In [5]:
# Specify the labels of the road segments in each route
all_routes = [[0, 1], [2, 3]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.5, 0.5]]*vehicle_limit))

### Topology setup

In [7]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

### Utility params setup

In [8]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 0

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go through these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 2min 20s, sys: 2.75 s, total: 2min 23s
Wall time: 2min 25s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]"
2,2,"[2, 3]",71.75,"[1, 1]","[0, 0]","[0, 0]"
3,3,"[2, 3]",75.0,"[2, 2]","[0, 0]","[0, 0]"
4,4,"[0, 1]",67.95,"[0, 0]","[0, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[2, 3]",170.9,"[74, 74]","[0, 0]","[0, 0]"
76,76,"[2, 3]",174.55,"[75, 75]","[0, 0]","[0, 0]"
77,77,"[0, 1]",68.15,"[73, 73]","[0, 0]","[0, 0]"
78,78,"[0, 1]",71.5,"[77, 77]","[0, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]",1
2,2,"[2, 3]",71.75,"[1, 1]","[0, 0]","[0, 0]",1
3,3,"[2, 3]",75.0,"[2, 2]","[0, 0]","[0, 0]",1
4,4,"[0, 1]",67.95,"[0, 0]","[0, 0]","[0, 0]",1
5,5,"[2, 3]",75.6,"[3, 3]","[0, 0]","[0, 0]",1
6,6,"[2, 3]",78.35,"[5, 5]","[0, 0]","[0, 0]",1
7,7,"[2, 3]",81.0,"[6, 6]","[0, 0]","[0, 0]",1
8,8,"[0, 1]",67.95,"[4, 4]","[0, 0]","[0, 0]",1
9,9,"[2, 3]",81.25,"[7, 7]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.read_csv("../2.2.3.1 Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0]","[0, 0]",[],5.153555,1.495229
1,1,"[0, 0]","[0, 0]",[],4.458580,1.059923
2,2,"[0, 0]","[0, 0]",[],4.909765,1.989908
3,3,"[0, 0]","[0, 0]",[],4.328507,0.968755
4,4,"[0, 0]","[0, 0]",[],5.917264,1.768378


## Summary of the 1st round

In [14]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,0,0.0,1,67.45
1,1,1,0.0,1,67.80
2,2,1,0.0,1,71.75
3,3,1,0.0,1,75.00
4,4,0,0.0,1,67.95
5,5,1,0.0,1,75.60
6,6,1,0.0,1,78.35
7,7,1,0.0,1,81.00
8,8,0,0.0,1,67.95
9,9,1,0.0,1,81.25


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[-32.9269487973401, 16.651092223153956]","[2.9412393065624364e-22, 1.0]",[0],5.153555,1.495229
1,1,"[16.651092223153956, -5.145362166819244]","[0.9999999996580834, 3.4191655826516596e-10]",[1],4.458580,1.059923
2,2,"[16.651092223153956, -332.7418801924899]","[1.0, 1.8220490500177464e-152]",[1],4.909765,1.989908
3,3,"[16.651092223153956, -3.6252776108621427]","[0.9999999984365483, 1.5634516939920461e-09]",[1],4.328507,0.968755
4,4,"[-127.50607720744732, 16.651092223153956]","[2.4736416770793185e-63, 1.0]",[0],5.917264,1.768378
5,5,"[16.651092223153956, -162.88814979627492]","[1.0, 1.0643829420466327e-78]",[1],4.100641,1.865665
6,6,"[16.651092223153956, -244.26367626801365]","[1.0, 4.854626220221188e-114]",[1],4.990654,1.906639
7,7,"[16.651092223153956, -35.519262933655526]","[1.0, 2.201418624905525e-23]",[1],6.014280,1.471883
8,8,"[-9.175035939451575, 16.651092223153956]","[6.079320642577591e-12, 0.9999999999939208]",[0],4.676816,1.200355
9,9,"[16.651092223153956, -43.75995114179145]","[1.0, 5.805200454429679e-27]",[1],5.827287,1.520083


In [15]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [16]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, 
                                all_routes, 
                                vehicle_limit, 
                                df, 
                                vehicle_preferences,
                                vehicle_rate, 
                                vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 1h 33min 4s, sys: 2min, total: 1h 35min 5s
Wall time: 1h 32min 25s
